In [ ]:
from langchain_community.graphs import Neo4jGraph
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI, OpenAI

import os
import textwrap

In [ ]:
# Load from environment
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

In [ ]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

In [ ]:
# Check the graph schema
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

In [ ]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What are the most popular movie categories
MATCH (m:Movie)-[:IN_CATEGORY]->(c:Category)
WITH c.name AS Category, COUNT(m) AS MoviesCount
RETURN Category, MoviesCount
ORDER BY MoviesCount DESC

# Find pairs of actors who have worked together in more than one movie
MATCH (p1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name < p2.name
WITH p1, p2, COUNT(m) AS sharedMovies
WHERE sharedMovies > 1
RETURN p1.name AS Actor1, p2.name AS Actor2, sharedMovies
ORDER BY sharedMovies DESC

# Get movies released in the United States, grouped by the release year
MATCH (m:Movie)-[:WHERE]->(c:Country {name: "United States"})
RETURN m.release_year AS ReleaseYear, collect(m.title) AS Movies
ORDER BY m.release_year

# Find all persons who have acted in at least one movie with Keanu Reeves
MATCH (p:Person {name: "Keanu Reeves"})-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(coActors)
WHERE p.name <> coActors.name
RETURN DISTINCT coActors.name AS CoActor

# Generate 10 movie recommendations for Stranger Things
MATCH (a:Movie {title:"Stranger Things"} )-[*2]-(b:Movie)
WHERE a <> b AND a.title < b.title
WITH DISTINCT a,b
RETURN a.title as title, b.title as recommendation, algo.linkprediction.adamicAdar(a, b) AS score
ORDER BY score DESC
LIMIT 10

The question is:
{question}"""

In [ ]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [ ]:
cypherChain = GraphCypherQAChain.from_llm(
    OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [ ]:
def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))

In [ ]:
prettyCypherChain("How many movies were released in 2012?")